In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
batch_size = 32
img_height = 180
img_width = 180

data_dir = "/Users/rohitjadhav/Desktop/Personal/nsfw-image-detection/clean_data"

In [3]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  directory=data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size
)

Found 800 files belonging to 4 classes.
Using 640 files for training.


In [4]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 800 files belonging to 4 classes.


Using 160 files for validation.


In [5]:
class_names = train_ds.class_names
print(class_names)

['hentai', 'neutral', 'porn', 'sexy']


In [6]:
num_classes = len(class_names)

model = tf.keras.Sequential([
  tf.keras.layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes)
])

In [7]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [8]:
epochs=10
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
)

Epoch 1/10


2024-01-26 10:18:15.138572: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


20/20 [==============================] - 10s 350ms/step - loss: 1.1640 - accuracy: 0.4453 - val_loss: 1.0123 - val_accuracy: 0.6500
Epoch 2/10
20/20 [==============================] - 9s 335ms/step - loss: 0.9933 - accuracy: 0.5625 - val_loss: 0.9784 - val_accuracy: 0.6250
Epoch 3/10
20/20 [==============================] - 10s 360ms/step - loss: 0.9280 - accuracy: 0.5656 - val_loss: 1.1409 - val_accuracy: 0.6125
Epoch 4/10
20/20 [==============================] - 9s 331ms/step - loss: 0.8515 - accuracy: 0.6109 - val_loss: 0.9036 - val_accuracy: 0.6500
Epoch 5/10
20/20 [==============================] - 11s 387ms/step - loss: 0.7348 - accuracy: 0.6703 - val_loss: 0.9797 - val_accuracy: 0.5188
Epoch 6/10
20/20 [==============================] - 10s 341ms/step - loss: 0.6279 - accuracy: 0.7219 - val_loss: 0.8221 - val_accuracy: 0.7125
Epoch 7/10
20/20 [==============================] - 10s 343ms/step - loss: 0.5666 - accuracy: 0.7172 - val_loss: 0.8029 - val_accuracy: 0.7188
Epoch 8/10
2

In [9]:
validation_loss, validation_accuracy = model.evaluate(val_ds)

print(f'Validation Accuracy: {validation_accuracy * 100:.2f}%')


5/5 [==============================] - 2s 49ms/step - loss: 1.0477 - accuracy: 0.6687
Validation Accuracy: 66.87%


In [10]:
test_folder = "/Users/rohitjadhav/Desktop/Personal/nsfw_img_detctn/test_imgs"

image_paths = [os.path.join(test_folder, file) for file in os.listdir(test_folder)]

images = [tf.keras.utils.load_img(image, target_size=(img_width, img_height)) for image in image_paths]

img_arrays = [tf.keras.utils.img_to_array(image) for image in images]

img_tensor = tf.concat([tf.expand_dims(img, 0) for img in img_arrays], axis=0)

predictions = model.predict(img_tensor)

class_probabilities = tf.nn.softmax(predictions, axis=1).numpy()

predicted_labels = np.argmax(class_probabilities, axis=1)

print(predicted_labels[0])

for i, image_path in enumerate(image_paths):
    class_name = class_names[predicted_labels[i]]
    confidence = 100 * np.max(class_probabilities[i])
    print("Image: {}, Predicted Class: {}, Confidence: {:.2f}%".format(image_path, class_name, confidence))


[[[ 21.  24.  35.]
  [ 20.  23.  33.]
  [ 22.  25.  36.]
  ...
  [ 74.  82.  98.]
  [ 76.  85. 100.]
  [ 44.  55.  70.]]

 [[ 17.  19.  30.]
  [ 17.  19.  30.]
  [ 18.  21.  31.]
  ...
  [ 47.  55.  73.]
  [101. 110. 130.]
  [ 61.  72.  89.]]

 [[ 16.  21.  33.]
  [ 19.  22.  32.]
  [ 18.  21.  31.]
  ...
  [ 51.  60.  80.]
  [103. 111. 131.]
  [ 68.  79.  96.]]

 ...

 [[ 12.  16.  28.]
  [  9.  14.  26.]
  [  7.  12.  24.]
  ...
  [ 17.  22.  34.]
  [ 15.  20.  32.]
  [ 15.  20.  32.]]

 [[  9.  14.  26.]
  [  9.  14.  26.]
  [  9.  14.  26.]
  ...
  [ 16.  21.  33.]
  [ 12.  16.  28.]
  [ 15.  20.  32.]]

 [[ 13.  18.  30.]
  [ 10.  15.  27.]
  [ 10.  15.  27.]
  ...
  [ 16.  21.  33.]
  [ 10.  15.  27.]
  [ 10.  15.  27.]]]
tf.Tensor(
[[[ 21.  24.  35.]
  [ 20.  23.  33.]
  [ 22.  25.  36.]
  ...
  [ 74.  82.  98.]
  [ 76.  85. 100.]
  [ 44.  55.  70.]]

 [[ 17.  19.  30.]
  [ 17.  19.  30.]
  [ 18.  21.  31.]
  ...
  [ 47.  55.  73.]
  [101. 110. 130.]
  [ 61.  72.  89.]]

 [[ 16.

In [1]:
model.save('../models/nsfw_model.h5')

NameError: name 'model' is not defined

In [33]:
sunflower_url = "https://www.google.com/url?sa=i&url=https%3A%2F%2Fthecinemaholic.com%2Fupcoming-adult-anime%2F&psig=AOvVaw2BdWUNz-_ZCOnb5lGr6a21&ust=1706010075604000&source=images&cd=vfe&opi=89978449&ved=0CBEQjRxqFwoTCNDUmbP18IMDFQAAAAAdAAAAABAI"
sunflower_path = tf.keras.utils.get_file('apple23', origin=sunflower_url)

img = tf.keras.utils.load_img(
    sunflower_path, target_size=(img_height, img_width)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x16b48dfd0>